In [2]:
# IMPORT ------------------------------------------------------------------------------------------

import os
import arcpy 
import numpy as np
import pandas as pd

from collections import Counter

pd.options.mode.chained_assignment = None  # default='warn'

# DEFINE PATHS ------------------------------------------------------------------------------------

base_path = os.path.abspath("..")
in_folder = os.path.join(base_path, "input")
in_MHN_GDB = os.path.join(in_folder, "MHN.gdb")

arcpy.env.workspace = in_MHN_GDB

## Highway

### Check if highway network is a multigraph

In [ ]:
fc = "hwynet/hwynet_arc"
fields = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] 
link_df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, fields), columns=fields)

link_df["AB"] = link_df["ANODE"].astype("string") + "-" + link_df["BNODE"].astype("string")
link_df = link_df[["AB", "ABB", "ANODE", "BNODE", "DIRECTIONS", "ROADNAME"]]
# link_df.AB.value_counts()
link_df_rev = link_df.copy()
link_df_rev["ANODE"] = link_df["BNODE"]
link_df_rev["BNODE"] = link_df["ANODE"]
link_df_rev["AB"] = link_df_rev["ANODE"].astype("string") + "-" + link_df_rev["BNODE"].astype("string") 

# ah nuts. It is a multigraph. 

# pd.merge(link_df, link_df_rev, how = "inner", on = "AB")

### Check for skeleton links which are later modified

In [75]:
fc = "hwyproj_coding"
fields = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] 
hwyproj_df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, fields), columns=fields)

fc = "hwyproj"
fields = [f.name for f in arcpy.ListFields(fc) if f.type!="Geometry"] 
year_df = pd.DataFrame(data=arcpy.da.SearchCursor(fc, fields), columns=fields)

hwyproj_df = pd.merge(hwyproj_df, year_df, how = "left", on = "TIPID")

In [85]:
skeleton = hwyproj_df[(hwyproj_df.ABB.str.endswith("0")) & (hwyproj_df.COMPLETION_YEAR != 9999) & (hwyproj_df.TOD == "0")]
skeleton_abbs = skeleton.ABB.value_counts().sort_values()

len(skeleton_abbs[skeleton_abbs >= 2])

27